In [3]:
import numpy as np
import pandas as pd

In [4]:
train_data=pd.read_csv('fraudTrain.csv',index_col=0)
test_data=pd.read_csv('fraudTest.csv',index_col=0)

In [5]:
train_data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,Hatch,UT,84735,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,Tuscarora,MD,21790,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,High Rolls Mountain Park,NM,88325,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,Manderson,SD,57756,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [6]:
data=pd.concat([train_data,test_data],axis=0)

In [7]:
data.shape

(1852394, 22)

In [8]:
data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,MO,63453,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,TX,77566,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,WA,99323,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,ID,83643,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [9]:
data[data.duplicated()]

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1852394 entries, 0 to 555718
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   first                  object 
 6   last                   object 
 7   gender                 object 
 8   street                 object 
 9   city                   object 
 10  state                  object 
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    object 
 16  dob                    object 
 17  trans_num              object 
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int64(5),

In [11]:
data.isnull().sum()

trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [12]:
data['trans_date_trans_time']=pd.to_datetime(data['trans_date_trans_time'])
data['day']=data['trans_date_trans_time'].dt.day
data['hour']=data['trans_date_trans_time'].dt.hour
data['month']=data['trans_date_trans_time'].dt.month

In [13]:
data.drop(columns=['trans_date_trans_time','first','last','job','street','city','state','zip','dob','zip','city_pop','trans_num','unix_time'],inplace=True)

In [14]:
from sklearn.preprocessing import LabelEncoder
cat_column=['category','merchant','gender']
encoders={}
for col in cat_column:
    le=LabelEncoder()
    data[col]=le.fit_transform(data[col])
    encoders[col]=le
    

In [15]:
data

,cc_num,merchant,category,amt,gender,lat,long,merch_lat,merch_long,is_fraud,day,hour,month
0,2703186189652095,514,8,4.97,0,36.0788,-81.1781,36.011293,-82.048315,0,1,0,1
1,630423337322,241,4,107.23,0,48.8878,-118.2105,49.159047,-118.186462,0,1,0,1
2,38859492057661,390,0,220.11,1,42.1808,-112.2620,43.150704,-112.154481,0,1,0,1
3,3534093764340240,360,2,45.00,1,46.2306,-112.1138,47.034331,-112.561071,0,1,0,1
4,375534208663984,297,9,41.96,1,38.4207,-79.4629,38.674999,-78.632459,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,30560609640617,507,5,43.77,1,40.4931,-91.8912,39.946837,-91.333331,0,31,23,12
555715,3556613125071656,264,7,111.84,1,29.0393,-95.4401,29.661049,-96.186633,0,31,23,12
555716,6011724471098086,496,7,86.88,0,46.1966,-118.9017,46.658340,-119.715054,0,31,23,12
555717,4079773899158,75,13,7.99,1,44.6255,-116.4493,44.470525,-117.080888,0,31,23,12


In [16]:
from geopy.distance import geodesic
def hav(lat1,long1,merch_lat2,merch_long2):
    return np.array([geodesic((a,b),(c,d)).km for  a,b,c,d in zip(lat1,long1,merch_lat2,merch_long2)])
data['distance']=hav(data['lat'],data['long'],data['merch_lat'],data['merch_long'])    

In [17]:
data.head(50)

,cc_num,merchant,category,amt,gender,lat,long,merch_lat,merch_long,is_fraud,day,hour,month,distance
0,2703186189652095,514,8,4.97,0,36.0788,-81.1781,36.011293,-82.048315,0,1,0,1,78.773821
1,630423337322,241,4,107.23,0,48.8878,-118.2105,49.159047,-118.186462,0,1,0,1,30.216618
2,38859492057661,390,0,220.11,1,42.1808,-112.2620,43.150704,-112.154481,0,1,0,1,108.102912
3,3534093764340240,360,2,45.00,1,46.2306,-112.1138,47.034331,-112.561071,0,1,0,1,95.685115
4,375534208663984,297,9,41.96,1,38.4207,-79.4629,38.674999,-78.632459,0,1,0,1,77.702395
5,4767265376804500,607,2,94.63,0,40.3750,-75.2045,40.653382,-76.152667,0,1,0,1,86.097358
6,30074693890476,534,3,44.54,0,37.9931,-100.9893,37.162705,-100.153370,0,1,0,1,118.094855
7,6011360759745864,107,2,71.65,1,38.8432,-78.6003,38.948089,-78.540296,0,1,0,1,12.754714
8,4922710831011201,250,9,4.27,0,40.3359,-79.6607,40.351813,-79.958146,0,1,0,1,25.333883
9,2720830304681674,563,4,198.39,0,36.5220,-87.3490,37.179198,-87.485381,0,1,0,1,73.939714


In [18]:
data['is_fraud'].value_counts()

is_fraud
0    1842743
1       9651
Name: count, dtype: int64

In [19]:
X=data.drop(columns=['is_fraud'])
y=data[['is_fraud']]

In [20]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=42)
X_resample,y_resample=smote.fit_resample(X,y)

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_resample,y_resample,test_size=0.2)

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth=2)
model=rf.fit(X_train,y_train)

c:\Users\vinot\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [23]:
y_pred=model.predict(X_test)
from sklearn.metrics import accuracy_score
print(f'{accuracy_score(y_test,y_pred)*100:.2f}%')

86.08%


In [ ]:
import joblib
joblib.dump(model, 'model.pkl')
joblib.dump(encoders['category'], 'label_encoder_category.pkl')
joblib.dump(encoders['merchant'], 'label_encoder_merchant.pkl')
joblib.dump(encoders['gender'], 'label_encoder_gender.pkl')


['label_encoder_gender.pkl']